In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 1000
eval_iters = 250

cuda


In [10]:
with open('bismarck_and_the_foundation_of_the_german_empire.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', 'â', 'ä', 'æ', 'è', 'é', 'ï', 'ô', 'ö', 'ü']
88


In [11]:
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([ 0, 24, 31, 41, 35, 23, 40, 25, 33,  9,  0,  0, 25, 30, 23, 38, 42, 27,
        40,  1, 31,  9,  0,  0, 24, 31, 40, 42, 30,  1, 23, 36, 26,  1, 38, 23,
        40, 27, 36, 42, 23, 29, 27,  9,  0,  0,  0, 37, 71, 71, 66,  1, 27, 55,
        72, 52, 69, 55,  1, 34, 56, 66, 67, 66, 63, 55,  1, 44, 66, 65,  1, 24,
        60, 70, 64, 52, 69, 54, 62,  1, 74, 52, 70,  1, 53, 66, 69, 65,  1, 52,
        71,  1, 71, 59, 56,  1, 64, 52, 65, 66])


In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)


inputs:
tensor([[58, 69, 60, 71, 76,  0, 66, 57],
        [ 1, 70, 56, 56, 65,  7,  1, 59],
        [ 1, 59, 60, 70,  1, 73, 60, 58],
        [59, 56,  1, 56, 65, 58, 52, 58]], device='cuda:0')
targets:
tensor([[69, 60, 71, 76,  0, 66, 57,  1],
        [70, 56, 56, 65,  7,  1, 59, 56],
        [59, 60, 70,  1, 73, 60, 58, 66],
        [56,  1, 56, 65, 58, 52, 58, 56]], device='cuda:0')


In [13]:
@torch.no_grad
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indices in the current content
        for _ in range(max_new_tokens):
            #get the predictions
            logits , loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :]#become (B, C)
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) #(B, C)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)#(B, 1)
            #append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)#(B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



h£jHVK3OHz[[;ænHJRpCs£XAeMâs_æsyzyyAèx5ïcylUh£XYj[.]!èIe6Yw4sydvcvè£zs);,gdjlWCwT7RNVe!äG!p8ïG7Hr;;t:_a[Y]vâtKU7M.k]S5J077Ftl?-æI6Dwü
AG_I7X2)éPIeWölnïo6U"Oô4N:)AYGKT PGhrd_]Ad5_èm£8sImu-HvnQhEïæf.".,KdVZt[WöoU370cZ6FBEmPtäsYo7?Xr.iPè 6ts£-aHhF]O1I7K]AJUyEyq1wæâ'E][uM7q8u4!öâ]U6XX 1WW1b7nä[V8WöAC.KFrsLæ.'ESäc1[HJmy
hct8Jic0G4A[dBét 0Lél5VZi"'(é90c5a-vNJ:P(é0ä[öôw7P"0c.je7(£ VKakSW:6a"48l!c£Xgwâ8 yivtèAeyVvi2"I?hqéS;âtX][U6Z;rUôs£ou4)61'Llx?-T3Rxâä;ctxfnAæ.p MzBcö:hjCt"£m8EY);)qä[;JïI-£!KéM,DzY)s


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses["train"]:.4f}, eval loss: {losses["val"]:.4f}')
    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.9804, eval loss: 5.0148
step: 250, train loss: 4.9378, eval loss: 4.9482
step: 500, train loss: 4.8839, eval loss: 4.8996
step: 750, train loss: 4.7992, eval loss: 4.8096
4.863898754119873


In [16]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


eh£æ"A(ô1-tRu?HMDKBQ".:ü9f)gäigéKtvysFle;7!QK)6b02O)VFnwIaæä!R]QPwQö[?Z,6bUj7jugô52EdJz5Bmlra(I"oü'D 6ZJn[Wi]vL'EU_1buh£-h9hubP7Fx9ï6FY.P7JDïzsyzü[ZH!wyCt[[PæfqèQZx?56FVYKcvqWöH!2tOKcO)éY[Y((H!üâäïg5-æ) 6WF[a:PU74äln;6aæD8.KFrsyyütiH6Wp5[Wpud39ä8'E(EoüwmU!n
h1f?ZX1T4?,60[:0üwKcy;cGJjHâXekdïaxAS".v8u3£AmPnö[WGFMj'C;h9O);vqTu9ïïC?cyiIQPINJQ.o02W8,Ok?WJACrdL
péJVææQi]üwWWf7O))éM((!O9.JD8VZjl!KèK)IèQBäD-t 5è".2Yt[u7qrwNô!wD7(dm4HgnèUY],?6S?ügö1Ku!nA)[slicæ büYï
EexxWöWæphè.£cHLu;,.ôl'E HHä MHJg0Gn-h
